In [9]:
# Import necessary libraries for environment, agents, HTTP requests, and typing
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from typing import Dict
import os
import requests
import asyncio

# Load environment variables from .env file
# override=True ensures existing variables are updated if present
load_dotenv(override=True)

True

In [10]:
# Define instructions for the triage agent to categorize customer queries
triage_instructions = """You are a customer support triage agent for a software company.
Your main goal is to categorize incoming customer queries into one of the following:
- 'technical_issue'
- 'billing_inquiry'
- 'feature_request'
- 'general_question'
You should also extract the key details of the customer's problem or request.
Your response should always start with the category, followed by a summary of the issue.
"""

# Create the triage agent with specified instructions and model
triage_agent = Agent(
    name="Triage Agent",
    instructions=triage_instructions,
    model="gpt-4o-mini"
)

In [11]:
# Define a sample customer query for testing the triage agent
customer_query = "My application keeps crashing after the latest update, and I can't log in at all. I need help urgently!"

# Process the query using the triage agent and print the result
print("Triage Agent processing query:")
result = await Runner.run(triage_agent, input=customer_query)
print(result.final_output)

Triage Agent processing query:
**Category:** technical_issue  
**Summary:** The customer is experiencing application crashes after a recent update and is unable to log in. They require urgent assistance.


In [12]:
import os
from typing import Dict
from agents import function_tool # Ensure this import is available
from twilio.rest import Client # Import Twilio client

@function_tool
def send_whatsapp_message(to_number: str, message_content: str) -> Dict[str, str]:
    """
    Sends a message to a WhatsApp number using the Twilio API.
    Args:
        to_number (str): The recipient's WhatsApp number (e.g., "+1234567890").
                         Must include the country code and be formatted correctly.
        message_content (str): The content of the message to send.
    Returns:
        Dict[str, str]: A dictionary indicating the status of the message sending.
    """
    account_sid = os.environ.get('WHATSAPP_ACCOUNT_SID')
    auth_token = os.environ.get('WHATSAPP_AUTH_TOKEN')
    from_number = os.environ.get('WHATSAPP_FROM_NUMBER')

    if not all([account_sid, auth_token, from_number]):
        return {"status": "error", "message": "WhatsApp API credentials (ACCOUNT_SID, AUTH_TOKEN, FROM_NUMBER) not found in environment variables. Please check your .env file."}

    try:
        client = Client(account_sid, auth_token)
        message = client.messages.create(
            from_=f'whatsapp:{from_number}', # Twilio requires 'whatsapp:' prefix
            body=message_content,
            to=f'whatsapp:{to_number}'       # Twilio requires 'whatsapp:' prefix
        )
        return {"status": "success", "sid": message.sid, "message": f"Message sent to WhatsApp successfully. SID: {message.sid}"}
    except Exception as e:
        return {"status": "error", "message": f"Failed to send WhatsApp message: {e}"}

# Inspect the created tool to verify its configuration
print(send_whatsapp_message)

FunctionTool(name='send_whatsapp_message', description='Sends a message to a WhatsApp number using the Twilio API.\nArgs:\n    to_number (str): The recipient\'s WhatsApp number (e.g., "+1234567890").\n                     Must include the country code and be formatted correctly.\n    message_content (str): The content of the message to send.\nReturns:\n    Dict[str, str]: A dictionary indicating the status of the message sending.', params_json_schema={'properties': {'to_number': {'title': 'To Number', 'type': 'string'}, 'message_content': {'title': 'Message Content', 'type': 'string'}}, 'required': ['to_number', 'message_content'], 'title': 'send_whatsapp_message_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x105cb5da0>, strict_json_schema=True, is_enabled=True)


In [13]:
# Create an agent to send messages to a WhatsApp number using the send_whatsapp_message tool
whatsapp_reporter = Agent(
    name="WhatsApp Reporter",
    instructions="You are an agent responsible for reporting important messages to a WhatsApp number. You always use the 'send_whatsapp_message' tool. The user will provide the recipient's WhatsApp number and the message content.",
    tools=[send_whatsapp_message], # Changed tool
    model="gpt-4o-mini"
)

# Test the WhatsApp Reporter with a sample message
print("\nWhatsApp Reporter sending a message:")
# IMPORTANT: Replace with a real, consented WhatsApp number for testing
# This number will be parsed by the agent from the input string.
customer_whatsapp_number_for_test = "+4917612345678" # Example: Replace with your actual WhatsApp number

message_to_send_whatsapp = "This is a test message from the WhatsApp Reporter Agent."
result = await Runner.run(
    whatsapp_reporter,
    input=f"Send the following message to WhatsApp number {customer_whatsapp_number_for_test}: {message_to_send_whatsapp}"
)
print(result.final_output)


WhatsApp Reporter sending a message:
The message has been sent successfully to the WhatsApp number +4917612345678. Message SID: SM5d587d7d512c72520f248770fb470ea3.


In [15]:
# Define instructions for the technical support agent
technical_support_instructions = """You are a highly knowledgeable technical support agent for a software company.
Your goal is to provide clear, concise, and helpful solutions for technical issues.
Always include troubleshooting steps and potential workarounds.
"""

# Define instructions for the billing support agent
billing_support_instructions = """You are a polite and efficient billing support agent.
Your goal is to address billing inquiries with accuracy and empathy.
Provide information on common billing issues, payment options, or how to contact the billing department directly.
"""

# Create specialized support agents
technical_support_agent = Agent(
    name="Technical Support Agent",
    instructions=technical_support_instructions,
    model="gpt-4o-mini"
)

billing_support_agent = Agent(
    name="Billing Support Agent",
    instructions=billing_support_instructions,
    model="gpt-4o-mini"
)

# Convert specialized agents into tools for the Support Manager
tech_tool = technical_support_agent.as_tool(
    tool_name="technical_support_response",
    tool_description="Generate a detailed technical support response for a given issue."
)

billing_tool = billing_support_agent.as_tool(
    tool_name="billing_support_response",
    tool_description="Generate a detailed billing support response for a given inquiry."
)

# Define all tools available to the Support Manager
manager_tools = [
    triage_agent.as_tool(tool_name="triage_query", tool_description="Categorize and summarize a customer query."),
    tech_tool,
    billing_tool,
    send_whatsapp_message # <--- CHANGED FROM send_discord_message
]

# Define instructions for the Support Manager to orchestrate the support process
support_manager_instructions = """You are a highly capable Support Manager for a software company.
Your primary role is to process incoming customer queries and send responses to the customer's WhatsApp number.
1. First, use the 'triage_query' tool to categorize the customer's message and summarize the core issue.
2. Based on the category provided by the 'triage_query' tool:
    - If the category is 'technical_issue', use the 'technical_support_response' tool to generate a technical solution.
    - If the category is 'billing_inquiry', use the 'billing_support_response' tool to generate a billing response.
    - For 'feature_request' or 'general_question', generate a polite acknowledgment and state that the request has been noted or a general answer.
3. Once a response is generated (either by a specialized agent tool or directly by you for general queries/feature requests), use the 'send_whatsapp_message' tool to send the final customer-facing response to the customer's WhatsApp number. The customer's WhatsApp number will be provided in the initial query (e.g., "My WhatsApp is +4917612345678, I have a billing issue..."). Extract this number and use it when calling the send_whatsapp_message tool.
"""

# Create the Support Manager agent
support_manager = Agent(
    name="Support Manager",
    instructions=support_manager_instructions,
    tools=manager_tools,
    model="gpt-4o-mini"
)

# Define sample customer queries for testing the support flow
# IMPORTANT: Embed the WhatsApp number in the query for the agent to extract
customer_query_1 = "My WhatsApp number is +4915222350056. I can't log in after the update, it says 'invalid credentials' even though I'm sure my password is correct. What should I do?"
customer_query_2 = "My WhatsApp number is +4915222350056. My last bill seems higher than usual, I'm not sure why. Can you check my subscription details?"
customer_query_3 = "My WhatsApp number is +4915222350056. I'd love to see a dark mode feature in your next update! It would be much easier on my eyes."

# Run the support flow for a technical issue
print("\n--- Running Support Flow for Technical Issue ---")
with trace("Automated Customer Support - Technical"):
    result_tech = await Runner.run(support_manager, customer_query_1)
    print(f"Final output for technical issue: {result_tech.final_output}")

# Run the support flow for a billing inquiry
print("\n--- Running Support Flow for Billing Inquiry ---")
with trace("Automated Customer Support - Billing"):
    result_billing = await Runner.run(support_manager, customer_query_2)
    print(f"Final output for billing inquiry: {result_billing.final_output}")

# Run the support flow for a feature request
print("\n--- Running Support Flow for Feature Request ---")
with trace("Automated Customer Support - Feature Request"):
    result_feature = await Runner.run(support_manager, customer_query_3)
    print(f"Final output for feature request: {result_feature.final_output}")


--- Running Support Flow for Technical Issue ---
Final output for technical issue: I have successfully sent your troubleshooting response to your WhatsApp number. Here’s a brief overview of the message:

- **Troubleshooting Steps**: Includes checking your credentials, resetting your password, clearing cache, and more.

If you need further assistance or have any questions, feel free to reach out!

--- Running Support Flow for Billing Inquiry ---
Final output for billing inquiry: I've sent a message to your WhatsApp number regarding your billing inquiry. Here's a brief overview of the response:

---

I understand that receiving a higher-than-usual bill can be concerning. Let’s look into your subscription details to clarify any discrepancies.

1. **Subscription Plan**: Please confirm your current plan so we can verify if there have been any changes or updates.
2. **Usage Charges**: Sometimes, additional charges may apply due to increased usage or added services.
3. **Promotions**: If you